#  second step replacement

In [1]:
from datasets import load_dataset #import public datasets
from tqdm import tqdm # for jupyter view
import csv #for handle csv type

In [2]:
import flair #NLP
from flair.data import Sentence #typcial class in flair stands for sentence
from flair.models import SequenceTagger #use for NER

# load tagger
tagger = SequenceTagger.load("flair/ner-english-large") # load a pre-trian NER moduel

2024-03-11 12:44:44,451 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [3]:
cls_data = load_dataset("imdb")
train_data = cls_data['train']
print(train_data[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
import faker
import pycountry
from faker import Faker
from random import choice

def replace_entities_flair_faker(text):
    # 初始化faker
    fake = Faker()

    # 创建sentence对象
    sentence = Sentence(text)

    # 使用预训练NER模型预测
    tagger.predict(sentence)

    replacements = []
    replacement_map = {}

    if not sentence.get_spans('ner'):
        return text

    for entity in sentence.get_spans('ner'):
        if entity.text in replacement_map:
            replacements.append((entity.start_position, entity.end_position, replacement_map[entity.text], entity.text))
            continue

        if entity.get_label().value == "ORG":
            if "company" in entity.get_label('ner').value.lower():
                # 公司
                repl = fake.company()
            elif "institution" in entity.get_label('ner').value.lower():
                # 机构
                repl = fake.company_suffix()
            else:
                # 其他组织
                repl = fake.catch_phrase()
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))
            replacement_map[entity.text] = repl

        elif entity.get_label().value == "PER":
            # 使用faker生成一个随机人名
            repl = fake.name()
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))
            replacement_map[entity.text] = repl

        elif entity.get_label().value == "LOC":
            if "city" in entity.get_label('ner').value.lower():
                # 城市
                repl = fake.city()
            elif "country" in entity.get_label('ner').value.lower():
                # 国家
                countries = list(pycountry.countries)
                repl = choice(countries).name
            else:
                # 其他地点
                repl = fake.address()
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))
            replacement_map[entity.text] = repl

    # 完成替换
    if replacements:
        res = []
        i = 0
        s = text
        for start, end, txt, orig in replacements:
            res.append(s[i:start] + txt)
            i = end
        res.append(s[end:])
        return ''.join(res)

    return text

In [5]:
text='It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is about a detective agency where the detectives and clients get romantically involved with each other. Five years later, Bogdanovich\'s ex-girlfriend, Cybil Shepherd had a hit television series called "Moonlighting" stealing the story idea from Bogdanovich. Of course, there was a great difference in that the series relied on tons of witty dialogue, while this tries to make do with slapstick and a few screwball lines.<br /><br />Bottom line: It ain\'t no "Paper Moon" and only a very pale version of "What\'s Up, Doc".'
src=replace_entities_flair_faker(text.replace("<br /><br />", " ").replace("<br />", ""))
src

'It was great to see some of my favorite stars of 30 years ago including John Vargas, Amanda Barnes and Madison Benitez. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing. Some of the smaller female roles were fine, Bernard Flores and Kimberly Howard were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Joan Cisneros got a chance to act in this her only important film role. The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Jim Brooks fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one. It is ironic that this movie is about a detective agency where the detectives

In [19]:
with open("data/flair/imdb_train_flair_faker.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        count = 0  # 初始化计数器
        src = replace_entities_flair_faker(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        writer.writerow((src, p['label']))
        count+=1
        if count>5:
            break

  1%|▏         | 342/25000 [04:55<5:54:41,  1.16it/s] 


KeyboardInterrupt: 

In [ ]:
cls_data = load_dataset("cnn_dailymail")
train_data = cls_data['train']
print(train_data[0])

In [ ]:
with open("data/flair/cnn_dm_train_flair_faker.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["article","highlights"])
    for p in tqdm(train_data):
        src = replace_entities_flair_faker(p['article'])
        trg = replace_entities_flair_faker(p['highlights'])
        writer.writerow((src, trg))

## Spacy faker

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [15]:
import spacy
import faker
import pycountry
from faker import Faker
from random import choice

# 加载spacy的英文模型和命名实体识别器
nlp = spacy.load("en_core_web_sm")

def replace_entities_spacy_faker(text):
    fake = Faker()  # 初始化faker

    doc = nlp(text)  # 使用spacy对文本进行处理
    replacements = []  # 存储需要替换的实体信息
    replacement_map = {}  # 存储已替换的实体映射

    for ent in doc.ents:
        if ent.text in replacement_map:  # 如果实体已被替换过
            replacements.append((ent.start_char, ent.end_char, replacement_map[ent.text], ent.text))
            continue  # 继续处理下一个实体

        # 根据实体类型生成替换文本
        if ent.label_ == "ORG":
            if "company" in ent.label_.lower():
                repl = fake.company()  # 公司
            elif "institution" in ent.label_.lower():
                repl = fake.company_suffix()  # 机构
            else:
                repl = fake.catch_phrase()  # 其他组织
        elif ent.label_ == "PERSON":
            repl = fake.name()  # 人名
        elif ent.label_ == "GPE":
            if "city" in ent.label_.lower():
                repl = fake.city()  # 城市
            elif "country" in ent.label_.lower():
                countries = list(pycountry.countries)
                repl = choice(countries).name  # 国家
            else:
                repl = fake.address()  # 其他地点
        else:
            continue  # 对其他类型实体不进行替换

        # 将替换信息添加到replacements和replacement_map
        replacements.append((ent.start_char, ent.end_char, repl, ent.text))
        replacement_map[ent.text] = repl

    if replacements:  # 如果有需要替换的实体
        res = []  # 存储替换后的文本
        start = 0
        for start_char, end_char, txt, orig in replacements:
            res.append(text[start:start_char] + txt)
            start = end_char
        res.append(text[start:])
        return ''.join(res)  # 将替换后的文本拼接并返回

    return text  # 如果没有需要替换的实体,返回原始文本

In [12]:
cls_data = load_dataset("imdb")
train_data = cls_data['train']
print(train_data[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [16]:
text='It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is about a detective agency where the detectives and clients get romantically involved with each other. Five years later, Bogdanovich\'s ex-girlfriend, Cybil Shepherd had a hit television series called "Moonlighting" stealing the story idea from Bogdanovich. Of course, there was a great difference in that the series relied on tons of witty dialogue, while this tries to make do with slapstick and a few screwball lines.<br /><br />Bottom line: It ain\'t no "Paper Moon" and only a very pale version of "What\'s Up, Doc".'
src=replace_entities_spacy_faker(text.replace("<br /><br />", " ").replace("<br />", ""))
src

'It was great to see some of my favorite stars of 30 years ago including Amber Watkins, Jonathan Moore and Maria Reyes. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing. Some of the smaller female roles were fine, Maria Luna and Karen Bailey were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Brenda Walker got a chance to act in this her only important film role. The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Scott Blackwell fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one. It is ironic that this movie is about a detective agency where the detectives an

In [18]:
with open("data/spacy/imdb_train_spacy_faker.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        src = replace_entities_spacy_faker(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        writer.writerow((src, p['label']))

  3%|▎         | 832/25000 [02:24<1:09:53,  5.76it/s]


KeyboardInterrupt: 

In [ ]:
with open("data/spacy/imdb_train_spacy_faker.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        src = replace_entities_spacy_faker(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        writer.writerow((src, p['label']))

In [ ]:
cls_data = load_dataset("cnn_dailymail")
train_data = cls_data['train']
print(train_data[0])

In [ ]:
with open("data/spacy/cnn_dm_train_spacy_faker.csv", "w",encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["article","highlights"])
    for p in tqdm(train_data):
        src = replace_entities_spacy_faker(p['article'])
        trg = replace_entities_spacy_faker(p['highlights'])
        writer.writerow((src, trg))